In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# Definitions
pd.set_option('display.float_format', lambda x: '%.3f' % x)

njobs = -1
randomState = 0


%matplotlib inline

In [2]:
#data from previous step
data_acc = pd.read_csv('data_acc.csv', dtype={'LSOA_of_Accident_Location': str})
data_cas = pd.read_csv('data_cas.csv')
data_mak = pd.read_csv('data_mak.csv')

In [3]:
data_acc.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Accident_Severity', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Date', 'Day_of_Week', 'Time',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'LSOA_of_Accident_Location', 'Class',
       'Date_month', 'Time_hour', 'weekdays'],
      dtype='object')

In [4]:
data_mak.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Journey_Purpose_of_Driver,Sex_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,make,Vehicle_age_group
0,2016010000005,1,8.000,0.000,6.000,0.000,0.000,0.000,0.000,0.000,...,1.000,1.000,8.000,nan,nan,nan,3.000,1.000,NaN,NaN
1,2016010000005,2,2.000,0.000,18.000,0.000,0.000,0.000,0.000,0.000,...,6.000,1.000,5.000,124.000,1.000,4.000,4.000,1.000,YAMAHA,0-5
2,2016010000006,1,9.000,0.000,18.000,0.000,2.000,2.000,10.000,1.000,...,6.000,2.000,7.000,1461.000,2.000,1.000,10.000,1.000,MERCEDES,0-5
3,2016010000008,1,9.000,0.000,18.000,0.000,4.000,5.000,0.000,0.000,...,6.000,1.000,5.000,1390.000,1.000,5.000,8.000,1.000,VOLKSWAGEN,0-5
4,2016010000016,1,11.000,0.000,18.000,0.000,1.000,0.000,0.000,0.000,...,1.000,1.000,8.000,6692.000,2.000,7.000,9.000,1.000,OTHER BRITISH,5-10


In [5]:
data_cas.head()

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile,Class_Cas
0,2016010000005,2,1,1,1.000,23.000,5.000,3,0.000,0.000,0.000,0.000,0.000,2,1.000,4.000,0
1,2016010000006,1,1,1,2.000,36.000,7.000,3,0.000,0.000,0.000,0.000,0.000,9,1.000,10.000,0
2,2016010000008,1,1,1,1.000,24.000,5.000,3,0.000,0.000,0.000,0.000,0.000,9,1.000,8.000,0
3,2016010000016,1,1,2,2.000,59.000,9.000,3,0.000,0.000,0.000,3.000,0.000,11,1.000,4.000,0
4,2016010000018,2,1,1,1.000,28.000,6.000,3,0.000,0.000,0.000,0.000,0.000,1,1.000,6.000,0


In [6]:
#I will include a pedestrian factor, since pedestrian circunstances are related to my hypothesis.
data_cas['include_pedestrian'] = data_cas['Casualty_Class'].map(lambda x: 1 if x==3 else 0)

In [7]:
cas_mak = pd.merge(data_cas, data_mak, on= ['Accident_Index', 'Vehicle_Reference'], how = 'left')

In [8]:
data = pd.merge(cas_mak,data_acc, on = 'Accident_Index', how = 'left')

In [9]:
unique_index = data['Accident_Index'].nunique()
unique_index

136621

In [10]:
data = data.set_index('Accident_Index')

data.columns

Index(['Vehicle_Reference', 'Casualty_Reference', 'Casualty_Class',
       'Sex_of_Casualty', 'Age_of_Casualty', 'Age_Band_of_Casualty',
       'Casualty_Severity', 'Pedestrian_Location', 'Pedestrian_Movement',
       'Car_Passenger', 'Bus_or_Coach_Passenger',
       'Pedestrian_Road_Maintenance_Worker', 'Casualty_Type',
       'Casualty_Home_Area_Type', 'Casualty_IMD_Decile', 'Class_Cas',
       'include_pedestrian', 'Vehicle_Type', 'Towing_and_Articulation',
       'Vehicle_Manoeuvre', 'Vehicle_Location-Restricted_Lane',
       'Junction_Location', 'Skidding_and_Overturning',
       'Hit_Object_in_Carriageway', 'Vehicle_Leaving_Carriageway',
       'Hit_Object_off_Carriageway', '1st_Point_of_Impact',
       'Was_Vehicle_Left_Hand_Drive', 'Journey_Purpose_of_Driver',
       'Sex_of_Driver', 'Age_Band_of_Driver', 'Engine_Capacity_(CC)',
       'Propulsion_Code', 'Age_of_Vehicle', 'Driver_IMD_Decile',
       'Driver_Home_Area_Type', 'make', 'Vehicle_age_group',
       'Location_Easting_

In [11]:
data = data.drop(['Vehicle_Reference', 'Casualty_Reference', 'Sex_of_Casualty', 'Age_of_Casualty', 
                  'Age_Band_of_Casualty',
                  'Casualty_Severity', 'Date', 'Car_Passenger', 'Bus_or_Coach_Passenger',
                  'Pedestrian_Road_Maintenance_Worker',
                 'Local_Authority_(District)', 'Local_Authority_(Highway)', 
                  'Accident_Severity', 'Towing_and_Articulation','Vehicle_Manoeuvre',
                  'Vehicle_Location-Restricted_Lane',
                  'Skidding_and_Overturning','Hit_Object_in_Carriageway','Vehicle_Leaving_Carriageway',
                  'Hit_Object_off_Carriageway', '1st_Point_of_Impact', 'Was_Vehicle_Left_Hand_Drive',
                  'Engine_Capacity_(CC)','Date','Propulsion_Code','Driver_Home_Area_Type', 'make',
                  'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 'Date_month',
                  'LSOA_of_Accident_Location',
    
                  'Casualty_Home_Area_Type'], axis = 1)

In [12]:
data.shape

(181384, 41)

In [13]:
#get rid of something else
data = data.drop(['Vehicle_Type','Casualty_Class', 'Pedestrian_Location', 'Pedestrian_Movement',
                  'Casualty_Type',
                  'Class_Cas', 'Junction_Location', 'Vehicle_age_group',
                  'Journey_Purpose_of_Driver', 'Sex_of_Driver', 'Junction_Control'], axis = 1)


In [14]:
data.shape

(181384, 30)

In [15]:
data.columns

Index(['Casualty_IMD_Decile', 'include_pedestrian', 'Age_Band_of_Driver',
       'Age_of_Vehicle', 'Driver_IMD_Decile', 'Location_Easting_OSGR',
       'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Day_of_Week', '1st_Road_Class',
       '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Class', 'Time_hour', 'weekdays'],
      dtype='object')

In [17]:
wea = data.groupby('Accident_Index', as_index=True).agg({'Casualty_IMD_Decile' : 'mean', 
                                                   'Age_Band_of_Driver' : 'mean', 'Driver_IMD_Decile': 'mean',
                                                  'Age_of_Vehicle' : 'max', 'include_pedestrian' : 'max'})


In [18]:
#merge with accidents table

#data_acc = data_acc.set_index('Accident_Index')
new_data_acc = data_acc.drop([ 'Junction_Control','Local_Authority_(District)', 'Local_Authority_(Highway)', 
                  'Accident_Severity','Date',
                  'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 'Date_month',], axis = 1)

In [19]:
data_acc.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Accident_Severity', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Date', 'Day_of_Week', 'Time',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'LSOA_of_Accident_Location', 'Class',
       'Date_month', 'Time_hour', 'weekdays'],
      dtype='object')

In [20]:
data = pd.merge(wea, new_data_acc, on='Accident_Index')

In [21]:
data = data.set_index('Accident_Index')

In [22]:
data.columns

Index(['Casualty_IMD_Decile', 'Age_Band_of_Driver', 'Driver_IMD_Decile',
       'Age_of_Vehicle', 'include_pedestrian', 'Location_Easting_OSGR',
       'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Day_of_Week', '1st_Road_Class',
       '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'LSOA_of_Accident_Location', 'Class',
       'Time_hour', 'weekdays'],
      dtype='object')

In [23]:
data.shape

(136621, 31)

In [24]:
#into groups
data['Casualty_IMD_Decile'] = np.round(data['Casualty_IMD_Decile'], 0)
data['Age_Band_of_Driver'] = np.round(data['Age_Band_of_Driver'], 0)
data['Driver_IMD_Decile'] = np.round(data['Driver_IMD_Decile'], 0)


In [25]:
#group IMD, less levels will show  the trends in a better way
bins = [0,3,6,8,10]
groups = ['low','med_low','med_high','high']
data['Casualty_IMD_Group'] = pd.cut(data['Casualty_IMD_Decile'],bins,labels = groups)
data['Casualty_IMD_Group'].value_counts()
data = data.drop(['Casualty_IMD_Decile'], axis = 1)

In [26]:
#group here
bins = [0,3,6,8,10]
groups = ['low','med_low','med_high','high']
data['Driver_IMD_Group'] = pd.cut(data['Driver_IMD_Decile'],bins,labels = groups)
data['Driver_IMD_Group'].value_counts()
data = data.drop(['Driver_IMD_Decile'], axis = 1)

In [27]:
#group vehicle age
bins = [0,5,10,15,99]
groups = ['0-5','5-10','10-15','+15']
data['Vehicle_Age_Group'] = pd.cut(data['Age_of_Vehicle'],bins,labels = groups)
data['Vehicle_Age_Group'].value_counts()
data = data.drop(['Age_of_Vehicle'], axis = 1)

In [28]:
#group here
bins = [0,1,2,3,99]
groups = ['1','2','3','+4']
data['Number_Vehicles_Group'] = pd.cut(data['Number_of_Vehicles'],bins,labels = groups)
data['Number_Vehicles_Group'].value_counts()
data = data.drop(['Number_of_Vehicles'], axis = 1)

In [29]:
#group here
bins = [0,1,2,3,4,1000]
groups = ['1','2','3','4','+4']
data['Number_Casualties_Group'] = pd.cut(data['Number_of_Casualties'],bins,labels = groups)
data['Number_Casualties_Group'].value_counts()
data = data.drop(['Number_of_Casualties'], axis = 1)

In [30]:
#group if junction, if roundabout or other
data['Junction_Detail'].value_counts()

0.000    56401
3.000    42755
6.000    13193
1.000    11740
8.000     4268
9.000     3584
5.000     1950
2.000     1700
7.000      924
Name: Junction_Detail, dtype: int64

In [31]:
data['Junction_Group'] = data['Junction_Detail'].map(lambda x: 1 if x==0  else  2 if x==1 or x==2  else 3 )
data = data.drop(['Junction_Detail'], axis = 1)

In [32]:
#grpup if pedestrian have some facilities or not
data['Pedestrian_Control'] = data['Pedestrian_Crossing-Human_Control'].map(lambda x: 1 if x==0  else  2)
data = data.drop(['Pedestrian_Crossing-Human_Control'], axis = 1)
data['Pedestrian_Control'].value_counts()

1    135415
2      1206
Name: Pedestrian_Control, dtype: int64

In [33]:
#same as before
data['Pedestrian_PhisFac'] = data['Pedestrian_Crossing-Physical_Facilities'].map(lambda x: 1 if x==0  else  2)
data['Pedestrian_PhisFac'].value_counts()
data = data.drop(['Pedestrian_Crossing-Physical_Facilities'], axis = 1)

In [34]:
#I have grouped this considering avaiability of light
data['Active_Light'] = data['Light_Conditions'].map(lambda x: 0 if x==5 or x==6 or x==7  else  1 )
data = data.drop(['Light_Conditions'], axis = 1)


In [35]:
#good or bad
data['Weather'] = data['Weather_Conditions'].map(lambda x: 1 if x==1  else  0 )
data = data.drop(['Weather_Conditions'], axis = 1)
data.Weather.value_counts()

1    112241
0     24380
Name: Weather, dtype: int64

In [36]:
#good or bad
data['Road_Surf_Cond'] = data['Road_Surface_Conditions'].map(lambda x: 1 if x==1  else  0 )
data = data.drop(['Road_Surface_Conditions'], axis = 1)


In [37]:
#good or bad
data['Special_Conds'] = data['Special_Conditions_at_Site'].map(lambda x: 1 if x==0  else  0 )
data = data.drop(['Special_Conditions_at_Site'], axis = 1)

data.Special_Conds.value_counts()

1    133269
0      3352
Name: Special_Conds, dtype: int64

In [38]:
#good or bad
data['Carriageway_Haz'] = data['Carriageway_Hazards'].map(lambda x: 1 if x==0  else  0 )
data = data.drop(['Carriageway_Hazards'], axis = 1)



In [39]:
data.Urban_or_Rural_Area.value_counts()

1    88468
2    48146
3        7
Name: Urban_or_Rural_Area, dtype: int64

In [40]:
data.weekdays.value_counts()

weekday    103779
weekend     32842
Name: weekdays, dtype: int64

In [41]:
#divide hour of the day in commute-non-commute 


data['Commute_hours'] = data['Time_hour'].map(lambda x: 1 if x in (7,9) or x in (15,17) else  0 )
#data = data.drop(['Carriageway_Hazards'], axis = 1)
data = data.drop(['Time_hour'], axis = 1)


In [42]:
#when no junction it is nan, in road number is 0, I will fillna with 0
data['2nd_Road_Class'] = data['2nd_Road_Class'].fillna(0)

In [43]:
data.shape

(136621, 31)

In [44]:
data.isna().sum()

Age_Band_of_Driver            6670
include_pedestrian               0
Location_Easting_OSGR            7
Location_Northing_OSGR           7
Longitude                        7
Latitude                         7
Day_of_Week                      0
1st_Road_Class                   0
1st_Road_Number                  0
Road_Type                        1
Speed_limit                     37
2nd_Road_Class                   0
2nd_Road_Number                617
Urban_or_Rural_Area              0
LSOA_of_Accident_Location     8351
Class                            0
weekdays                         0
Casualty_IMD_Group           25586
Driver_IMD_Group             28030
Vehicle_Age_Group            38062
Number_Vehicles_Group            0
Number_Casualties_Group          0
Junction_Group                   0
Pedestrian_Control               0
Pedestrian_PhisFac               0
Active_Light                     0
Weather                          0
Road_Surf_Cond                   0
Special_Conds       

In [45]:
data = data.dropna()
data.shape

(77703, 31)

In [46]:
data.to_csv('data_to_model.csv')

In [173]:
(data[data.Class == 0].count()[0]/data.shape[0])*100

83.48068929127575